# Create Revit 2025 Type Parameters Activity

This notebook creates an activity for Revit 2025 using the Autodesk Platform Services (APS) Design Automation API to add type parameters from JSON configuration.

## Step 1: Setup and Import Libraries

Import all necessary libraries and modules.

In [26]:
import os
import json
import logging
import uuid
from pathlib import Path
from dotenv import load_dotenv

from aps_automation_sdk.classes import (
    Activity,
    ActivityInputParameter,
    ActivityOutputParameter,
    ActivityJsonParameter,
    AppBundle,
    WorkItem
)

from aps_automation_sdk.utils import (
    delete_activity,
    delete_appbundle,
    get_token,
    set_nickname
)

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

## Step 2: Load Environment Variables and Configure Authentication

Load credentials from environment variables and get an authentication token.

**Prerequisites:**
1. Create an APS application to get your credentials. Follow these tutorials:
   - Create App: https://aps.autodesk.com/en/docs/oauth/v2/tutorials/create-app
   - Design Automation Setup: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step2-create-forge-app/

2. Create a `.env` file in the project root with your credentials (see `.env.sample` for reference)

**Note on Nicknames:** The nickname must be unique across all APS apps globally. If you encounter errors, try using alphanumeric unique values. Choose a distinctive nickname to avoid conflicts.

In [27]:
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")

# Get authentication token
token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

# Set nickname. If the app already has a nickname, the previous one will be returned.
nickname = set_nickname(token, "myUniqueNickNameHere")

print(f"Authentication successful. Nickname: {nickname}")

Authentication successful. Nickname: viktortest


## Step 3: Define Revit 2025 Configuration

Set up constants for the Revit 2025 type parameters workflow.

**Note:** For other Revit versions, change the year suffix and engine version accordingly.

In [ ]:
# Define constants for Revit 2025
# For other versions, update the year suffix (2023, 2024, 2025, 2026)
year = 2025
app_bundle_base_name = "TypeParametersFromJsonDA"
activity_base_name = "TypeParametersActivity"
alias = "dev"
bucket_key = uuid.uuid4().hex

# Add year suffix to names
app_bundle_name = f"{app_bundle_base_name}{year}"
activity_name = f"{activity_base_name}{year}"

# Path to the bundle ZIP file (relative to this notebook's directory)
zip_path = Path.cwd() / "files" / "TypeParametersFromJsonDA.bundle.zip"

# Create full aliases
appbundle_full_alias = f"{nickname}.{app_bundle_name}+{alias}"
activity_full_alias = f"{nickname}.{activity_name}+{alias}"

print(f"Revit Version: {year}")
print(f"App Bundle: {appbundle_full_alias}")
print(f"Activity: {activity_full_alias}")
print(f"ZIP Path: {zip_path}")

App Bundle: viktortest.TypeParametersFromJsonDA2025+dev
Activity: viktortest.TypeParametersActivity2025+dev


## Step 4: Create and Deploy App Bundle

Register and upload the app bundle that contains the logic for creating and setting type parameters from JSON.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step4-publish-appbundle/

In [ ]:
bundle = AppBundle(
    appBundleId=app_bundle_name,
    engine="Autodesk.Revit+2025",  # For Revit 2023: Autodesk.Revit+2023, For 2024: Autodesk.Revit+2024, etc.
    alias=alias,
    zip_path=str(zip_path),
    description=f"Type parameters from JSON for Revit {year}",
)

bundle.deploy(token)
print(f"✓ App bundle deployed successfully for Revit {year}!")

App bundle deployed successfully!


## Step 5: Define Activity Parameters

Create input and output parameters for the activity.

**Note:** This example includes a JSON file as an input parameter. The JSON will be uploaded to OSS storage rather than embedded in the work item payload to avoid the 60KB size limitation.

**Reference:** https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/#step-1-create-a-new-activity

In [31]:
# Input: Revit file
input_revit = ActivityInputParameter(
    name="rvtFile",
    localName="input.rvt",
    verb="get",
    description="Input Revit model",
    required=True,
    is_engine_input=True,
    bucketKey=bucket_key,
    objectKey="input.rvt",
)

# Output: Result Revit file
output_file = ActivityOutputParameter(
    name="result",
    localName="result.rvt",
    verb="put",
    description="Result Revit model",
    bucketKey=bucket_key,
    objectKey="result.rvt",
)

# Input: JSON configuration for type parameters
input_json = ActivityJsonParameter(
    name="configJson",
    localName="revit_type_params.json",
    verb="get",
    description="Type parameter JSON configuration",
)

print("Activity parameters defined successfully!")

Activity parameters defined successfully!


## Step 6: Create and Deploy Activity

Create the activity that links the app bundle with the parameters.

**Note:** This wrapper covers all the API calls from the tutorial step: https://aps.autodesk.com/en/docs/design-automation/v3/tutorials/revit/step5-publish-activity/

In [ ]:
activity = Activity(
    id=activity_name,
    parameters=[input_revit, output_file, input_json],
    engine="Autodesk.Revit+2025",  # For Revit 2023: Autodesk.Revit+2023, For 2024: Autodesk.Revit+2024, etc.
    appbundle_full_name=appbundle_full_alias,
    description=f"Create type parameters from JSON for Revit {year}",
    alias=alias,
)

# Set the Revit command
activity.set_revit_command_line()
activity.deploy(token=token)
print(f"✓ Activity deployed successfully for Revit {year}!")

Activity deployed successfully!


## Step 7: Upload Input Files

Upload the Revit file and JSON configuration file to Object Storage Service (OSS).

**Note:** Input files should be in the `files/` subdirectory relative to this notebook.

In [ ]:
# Path to input Revit file (in files/ subdirectory)
input_rvt_path = Path.cwd() / "files" / "sample_model_2025.rvt"

if not input_rvt_path.exists():
    raise FileNotFoundError(f"Input Revit file not found: {input_rvt_path}")

input_revit.upload_file_to_oss(file_path=str(input_rvt_path), token=token)
print(f"✓ Input Revit file uploaded: {input_rvt_path.name}")

Input Revit file uploaded: sample_model_2025.rvt


In [ ]:
# Load type parameters configuration from JSON file (in files/ subdirectory)
json_config_path = Path.cwd() / "files" / "revit_type_params.json"

if not json_config_path.exists():
    raise FileNotFoundError(f"JSON configuration file not found: {json_config_path}")

with open(json_config_path, 'r') as f:
    type_params_config = json.load(f)

input_json.set_content(type_params_config)
print("✓ Type parameters configuration loaded:")
print(json.dumps(type_params_config, indent=2))

Type parameters configuration loaded:
{
  "ParameterName": "wombat",
  "ParameterGroup": "PG_DATA",
  "Targets": [
    {
      "TypeName": "CL_W1",
      "FamilyName": "Basic Wall",
      "Value": "95"
    }
  ]
}


## Step 8: Create and Execute Work Item

Create a work item with all parameters and execute it.

In [ ]:
work_item = WorkItem(
    parameters=[input_revit, output_file, input_json],
    activity_full_alias=activity_full_alias
)

print("Work item created. Starting execution...")
print("This may take several minutes...")

Work item created. Starting execution...


## Step 9: Poll Work Item Status

Monitor the work item execution until completion (max 10 minutes).

In [ ]:
status_resp = work_item.execute(token=token, max_wait=600, interval=10)
last_status = status_resp.get("status", "")

print("=" * 60)
print(f"Work item completed with status: {last_status}")
print("Full response:")
print(json.dumps(status_resp, indent=2))

2025-11-24 14:29:06,220 INFO Polling work item status, id=3e578ec17c3f49df970b5320123a4ba7
2025-11-24 14:29:07,074 INFO [  0s] status=inprogress report_url=None
2025-11-24 14:29:17,818 INFO [ 10s] status=inprogress report_url=None
2025-11-24 14:29:28,581 INFO [ 20s] status=inprogress report_url=None
2025-11-24 14:29:39,422 INFO [ 30s] status=inprogress report_url=None
2025-11-24 14:29:50,710 INFO [ 40s] status=success report_url=https://dasprod-store.s3.us-east-1.amazonaws.com/workItem/viktortest/3e578ec17c3f49df970b5320123a4ba7/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIF9RdxDxs7hKpxKuNC1i5Q27yuyO5bQJbWodffiCNFNxAiBR9LcWWPr64qhOkKYgZjGsjhwcLJ5k45tzM5XbLiZnmCqAAwhcEAQaDDIyMDQ3MzE1MjMxMCIMfoZKGOLd%2FiwHc0zRKt0CQe6jCxX86Q3WrhTQQ8IS3KRkuxI6OJes%2FjIKSmvQudXOWB5y9%2BSD1e9dGS8A586D%2FyC8ZSSib07XilugWicmjYuW9GA%2BEj3m3nn%2BjWJETprzkN%2Bi4mfvALQsxjTr8lLNInfEJWck94cWSOIRSmvC6GbU427HDDhX3h5RuJOTi4uss8wrKw6yfpXf7JL7TSmC

Work item completed with status: success
Full response: {
  "status": "success",
  "reportUrl": "https://dasprod-store.s3.us-east-1.amazonaws.com/workItem/viktortest/3e578ec17c3f49df970b5320123a4ba7/report?X-Amz-Expires=3600&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEJP%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIF9RdxDxs7hKpxKuNC1i5Q27yuyO5bQJbWodffiCNFNxAiBR9LcWWPr64qhOkKYgZjGsjhwcLJ5k45tzM5XbLiZnmCqAAwhcEAQaDDIyMDQ3MzE1MjMxMCIMfoZKGOLd%2FiwHc0zRKt0CQe6jCxX86Q3WrhTQQ8IS3KRkuxI6OJes%2FjIKSmvQudXOWB5y9%2BSD1e9dGS8A586D%2FyC8ZSSib07XilugWicmjYuW9GA%2BEj3m3nn%2BjWJETprzkN%2Bi4mfvALQsxjTr8lLNInfEJWck94cWSOIRSmvC6GbU427HDDhX3h5RuJOTi4uss8wrKw6yfpXf7JL7TSmCQLz2h8aBCftQ8QNBWktpXS2gmycpoAtPpPvcrCk7zF%2B%2BG59CNQk3CNKLcVYUTM3Ljx4x19n86WGrRBfgpCC300aCpqnhH3rSazUIupfb1cCDuytFJN5Mc0vBxpbEmzGKqmgn5b7ssM2qN79mCFlYCz%2BdqB762Gh153pT3yBKYVHtk1NA1kCNP83EkFJ1fRMJfKcenM%2By%2Bj8o8lfClZEA5jzMLj%2Bvnlfi47Pw3xKgtSGg6RE9Vr6W7d3IiXl5Y4OcXMEypdiuJpVv6N5UHzDyz5LJBjqfAVidF69DTNslQexeAufOU20dyIEEAb5T%2F7Dn63

## Step 10: Download Results

If successful, download the results to the `files/output/` subdirectory.

In [ ]:
if last_status == "success":
    print("\n" + "=" * 60)
    print("Downloading results...")
    out_dir = Path.cwd() / "files" / "output"
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / "output.rvt"
    
    output_file.download_to(output_path=str(out_file), token=token)
    print(f"✓ Download successful: {out_file}")
    print("\nType parameters workflow completed successfully!")
else:
    print(f"\n✗ Work item failed with status: {last_status}")
    print("Check the response above for error details.")

✅ Download successful: files/output/output.rvt


## Step 11: Cleanup Resources (Optional)

Delete the activity and app bundle to clean up resources.

In [ ]:
print("Cleaning up resources...")

# Delete activity
try:
    delete_activity(activityId=activity_name, token=token)
    print(f"✓ Deleted activity for Revit {year}: {activity_full_alias}")
except Exception as e:
    print(f"✗ Failed to delete activity for Revit {year}: {e}")

# Delete app bundle
try:
    delete_appbundle(appbundleId=app_bundle_name, token=token)
    print(f"✓ Deleted app bundle for Revit {year}: {appbundle_full_alias}")
except Exception as e:
    print(f"✗ Failed to delete app bundle for Revit {year}: {e}")

print("\nCleanup completed!")

Cleaning up: deleting activity and app bundle...
⚠️ Failed to delete activity: 404 Client Error: Not Found for url: https://developer.api.autodesk.com/da/us-east/v3/activities/TypeParametersActivity2025
⚠️ Failed to delete activity: 404 Client Error: Not Found for url: https://developer.api.autodesk.com/da/us-east/v3/activities/TypeParametersActivity2025
⚠️ Failed to delete app bundle: 404 Client Error: Not Found for url: https://developer.api.autodesk.com/da/us-east/v3/appbundles/TypeParametersFromJsonDA2025

🎉 Type parameters workflow completed!
⚠️ Failed to delete app bundle: 404 Client Error: Not Found for url: https://developer.api.autodesk.com/da/us-east/v3/appbundles/TypeParametersFromJsonDA2025

🎉 Type parameters workflow completed!


## Important Notes on File Paths

**Expected Directory Structure:**
```
2025 - 2026 - Net8/
  ├── create_activity_2025.ipynb (this notebook)
  ├── addin.cs
  └── files/
      ├── TypeParametersFromJsonDA.bundle.zip  # Compiled bundle (.NET 8)
      ├── revit_type_params.json               # Type parameters config
      ├── sample_model_2025.rvt                # Input Revit file
      └── output/
          └── output.rvt                       # Generated output
```

**To Adapt for Other Versions:**
1. Update the `year` variable in Step 3 (2023, 2024, 2025, or 2026)
2. Update the engine version in Steps 4 and 6 to match:
   - Revit 2023: `Autodesk.Revit+2023` (.NET 4.8)
   - Revit 2024: `Autodesk.Revit+2024` (.NET 4.8)
   - Revit 2025: `Autodesk.Revit+2025` (.NET 8)
   - Revit 2026: `Autodesk.Revit+2026` (.NET 8)
3. Ensure the correct bundle ZIP is in the `files/` directory for that version